In [1]:
from comet_ml import Experiment

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from random import choice
import urllib.request  # the lib that handles the url stuff
import time

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import zero_one_loss
from itertools import *


torch.cuda.empty_cache()

url = "https://raw.githubusercontent.com/gracexwho/drug-drug-interactions/master/ChCh-Miner_durgbank-chem-chem.tsv"
url_data = urllib.request.urlopen(url) 

G = nx.read_edgelist(url_data)

print(G.number_of_nodes())
print(G.number_of_edges())


# Create an experiment
experiment = Experiment(api_key="yeThLw8MLFuaMF3cVW1b9IsIt",
                        project_name="Node2Vec", workspace="gracexwho")

# Report any information you need by:

1514
48514


COMET INFO: old comet version (2.0.2) detected. current: 2.0.12 please update your comet lib with command: `pip install --no-cache-dir --upgrade comet_ml`
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/gracexwho/node2vec/7cdc075592c84e59af2a36fbd129868d



In [2]:


################# CONTROL ##################

# 0.1, 5000, 100, 5, 3

hyper_params = {"learning_rate": 0.05, "epochs": 10000, "num_walks": 1000, "walk_length": 5, "window_size": 3}
experiment.log_parameters(hyper_params)


################# CONTROL ##################

In [3]:
# Generate random walks
pairs = []

for i in range(hyper_params['num_walks']):
    current = choice(list(G.nodes()))
    walk = [current]
    y = []
    
    for w in range(hyper_params['walk_length']):
        # walk to an adjacent node
        # error: some adjacent nodes are NOT IN the training set
        c = list(G.adj[current])
        current = choice(c)
        walk.append(current)
    
    # take permutations as closely related within the window size
    y = [permutations(walk[i : i+hyper_params['window_size']], 2) for i in range(len(walk)-hyper_params['window_size'])]
    z = []
    for l in y:
        z.extend(list(l))
    pairs.extend(z)

# remove duplicates
pairs = list(dict.fromkeys(pairs))


class Encoder(nn.Module):
  # should return VECTORS for each node
    def __init__(self):
        super(Encoder, self).__init__()
        
        #self.dropout = nn.Dropout(p=0.2)
        # one layer, return embeds(node_ids) which is a long tensor
        #learnrate might be too big if doesn't decrease
        self.embed = nn.Embedding(G.number_of_nodes(), 64)

    
    def forward(self, x):
        # take the node name as input and 
        x = self.embed(x)
        return x

# embeds.weight = embeds.weight/np.sqrt(mbed_dim)
  # Loss function can't be a float, it should be a tensor
  # and also DON'T unwrap a tensor at any point, that gets rid of grad
  # Keep it in tensor operators: maybe change node_dict into a tensor


def common_neighbours(T, u, v):
    Unodes = list(T.adj[u])
    Vnodes = list(T.adj[v])
    matches = [x for x in Unodes if x in Vnodes]
    return iter(matches)

def jaccard_index(T, u, v):
    common = len(list(common_neighbours(T, u, v)))
    union = T.degree[u] + T.degree[v] - common
    if union == 0:
        return 0
    else:
        return (common/union)

def adamic_adar(T, u, v):
    common = common_neighbours(T, u, v)
    total = 0
    for c in common:
        total = total + 1/np.log(T.degree[c])
    return total

def CN(T, edges):
    CN = {}
    for (u,v) in edges:
        CN[(u,v)] = len(list(common_neighbours(T,u,v)))
    return CN.values()


def JC(T, edges):
    JC = {}
    for (u,v) in edges:
        JC[(u,v)] = jaccard_index(T, u, v)
    return JC.values()


def AA(T, edges):
    AA = {}
    for (u,v) in edges:
        AA[(u,v)] = adamic_adar(T, u, v)
    return AA.values()

print("done")

done


In [4]:
import random
## Always "Restart Kernel and Clear Output if you're going to train again"


id_map = {}



def get_id(id_string):
    if id_string in id_map.keys():
        return id_map[id_string]
    else:
        ID = len(id_map)
        id_map[id_string] = ID
        return ID
    
        
def process_batch(batch):
    left_ids = torch.cuda.LongTensor([pair[0] for pair in batch])
    right_ids = torch.cuda.LongTensor([pair[1] for pair in batch])
    neg_ids = torch.cuda.LongTensor([np.random.randint(0, G.number_of_nodes()) for _ in batch])
    
    #print(left_ids)
    left_embeds = model(left_ids)
    right_embeds = model(right_ids)
    neg_embeds = model(neg_ids)
    
    pos_score = torch.mm(torch.t(left_embeds), right_embeds)
    neg_score = torch.mm(torch.t(left_embeds), neg_embeds)
    
    loss = get_loss(pos_score, neg_score)
    return loss
    
                          
def get_loss(pos, neg):
    m = nn.Sigmoid()
    loss = -torch.mean(torch.log(m(pos))) - torch.mean(torch.log(1 - m(neg)))
    return loss



model = Encoder()
model.embed.weight.data = (model.embed.weight.data/np.sqrt(64))
model.to('cuda')

optimizer = optim.SGD(model.parameters(), lr=hyper_params['learning_rate'])

epochs = hyper_params['epochs']

for n in list(list(G.nodes())):
    get_id(n)

pairs = [(get_id(pair[0]) , get_id(pair[1])) for pair in pairs]


alpha = 0.9
train_loss = 0
ep = hyper_params['epochs']

for e in range(ep):
    random.shuffle(pairs)
    train_loss = 0
    batch_size = 64
    batch = []
    index=0
    # the reason you can't use training_loss for optimizer is because train_loss isn't defined within the while loop
    # try doing train_loss.back() OUTSIDE of while loop?
    
    while index+batch_size < len(pairs):
        batch = pairs[index:min(index+batch_size, len(pairs))]
        index += batch_size
                
        loss = process_batch(batch)
        train_loss = alpha * train_loss + (1-alpha)*loss
        
    optimizer.zero_grad() 
    train_loss.backward()        #change back to loss if retain_graph error
    optimizer.step()
        #print("loss", loss)
        
    if e % 500 == 0:
        print("Training loss: ", train_loss)
        

      
torch.save(model.state_dict(), 'node2vec.pth')
    
print("done")

Training loss:  tensor(1.3897, device='cuda:0', grad_fn=<AddBackward0>)
Training loss:  tensor(1.3904, device='cuda:0', grad_fn=<AddBackward0>)
Training loss:  tensor(1.3901, device='cuda:0', grad_fn=<AddBackward0>)
Training loss:  tensor(1.3899, device='cuda:0', grad_fn=<AddBackward0>)


KeyboardInterrupt: 

In [ ]:
# Split graph into training and validation set
import random


url = "https://raw.githubusercontent.com/gracexwho/drug-drug-interactions/master/ChCh-Miner_durgbank-chem-chem.tsv"
url_data = urllib.request.urlopen(url) 
T = nx.read_edgelist(url_data)
V = nx.Graph()

# WAIT is it because the EMBEDDINGS are based on the TRAINING GRAPH and then THAT'S REMOVED

num_val = 5000
num_neg = 5000


val_set = random.sample(list(T.edges()), num_val)
train_set = list(T.edges())
train_set = [e for e in train_set if e not in val_set]

#T.remove_edges_from(val_set)
#V.add_edges_from(val_set)

#T.remove_nodes_from(list(nx.isolates(T)))
# this removes nodes that don't have any neighbors from training graph

num_train = num_neg + len(train_set)


print("Number of validation edges", num_val)
print("Number of training edges", num_train)


In [ ]:
##################### TRAINING LOOP ######################
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import zero_one_loss


start_time = time.time()


class LogisticRegression(torch.nn.Module):
     
    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(64, 1)
        #self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred



train_edges = train_set
val_edges = val_set

y_label = [1]*len(train_edges)
neg = [0]*num_neg
y_label = y_label + neg
y_label = np.array(y_label)


negs = random.sample(list(nx.non_edges(T)), num_neg)
train_edges = train_edges + negs

link_model = LogisticRegression()
criterion = torch.nn.BCELoss(reduction='mean')         # this is already binary cross entropy!
optimizer_link = torch.optim.SGD(link_model.parameters(), lr = 0.03) 
weight = model.embed.weight.data * 10


left = [pair[0] for pair in train_edges]
right = [pair[1] for pair in train_edges]
left = [get_id(l) for l in left]
right = [get_id(r) for r in right]
left_ids = [weight[ids] for ids in left]
lf = torch.stack(left_ids)
right_ids = [weight[ids] for ids in right]
rg = torch.stack(right_ids)


#weight = torch.FloatTensor(np.random.rand(weight.shape[0], weight.shape[1]))

dot_prod = torch.bmm(lf.view(num_train, 1, 64), rg.view(num_train, 64, 1)) 
dot_prod = torch.squeeze(dot_prod)
dot_prod = dot_prod.cpu()
dotproduct = dot_prod.numpy()

score = roc_auc_score(y_label, dotproduct)
print("The auc score for dot product is: ", score)


link_model.to('cuda')


element_mul = (lf * rg)
#element_mul = (lf + rg)
element_mul = element_mul.cuda()
y_label_model = torch.cuda.FloatTensor(y_label)

print(element_mul[0])

link_model.train()


# Keep it at 5000, more is overfitting

ra = 50000
for e in range(ra):
    optimizer_link.zero_grad() 
    y_pred = link_model(element_mul)
    y_pred = torch.squeeze(y_pred)
    loss = criterion(y_pred, y_label_model)
    loss.backward() 
    optimizer_link.step() 
    auc = roc_auc_score(y_label, y_pred.cpu().detach().numpy())
    if e % 2000 == 0:
        print(auc)
        print(loss)
    
print("Here's the graph index value:")
test = list(AA(T, train_edges))
print(roc_auc_score(y_label, test))
    
torch.save(link_model.state_dict(), 'linkpred.pth')
        
print("--- %s minutes ---" % ((time.time() - start_time)//60))
    

# If you get CUDA error, just keep running again and it'll work

In [ ]:
########### VALIDATION LOOP ##############
y_hat = []
y_true = [1]*len(val_edges)
neg = [0]*num_val
y_true = y_true + neg
y_true = np.array(y_true)


negs = random.sample(list(nx.non_edges(V)), num_val)
val_edges = val_edges + negs


link_model.eval()

for (x,y) in val_edges:
    u = weight[get_id(x)]
    v = weight[get_id(y)]
    em = u * v
    em = em.cuda()
    pred = link_model(em)
    y_hat.append(pred.item())

print("done")

In [ ]:
accuracy = roc_auc_score(y_true, y_hat)

jaccard = list(JC(T, val_edges))
jaccard = roc_auc_score(y_true, jaccard)

adamic = list(AA(T, val_edges))
adamic = roc_auc_score(y_true, adamic)

common = list(CN(T, val_edges))
common = roc_auc_score(y_true, common)


print("The auc score for dot product is: ", score)
print("The auc score for elementwise mul: ", accuracy)
print("The auc score for JC index: ", jaccard)
print("The auc score for AA index: ", adamic)
print("The auc score for CN index: ", common)



In [ ]:

[fpr, tpr, thresh] = roc_curve(y_true, common)

ratioMax = tpr - fpr

fig1 = plt.figure()
common_line = plt.plot(fpr, tpr)
fig1.suptitle('AUC of Local Similarity Indices', fontsize=20)
plt.xlabel('FPR', fontsize=18)
plt.ylabel('TPR', fontsize=16)
plt.axis([0, 1.0, 0, 1.0])                      # set [xmin, xmax, ymin, ymax]


[fpr2, tpr2, thresh2] = roc_curve(y_true, jaccard)
ratioMax = tpr2 - fpr2
jaccard_line = plt.plot(fpr2, tpr2)


[fpr3, tpr3, thresh3] = roc_curve(y_true, adamic)
ratioMax = tpr3 - fpr3
adamic_line = plt.plot(fpr3, tpr3)


legend((common_line, jaccard_line, adamic_line), ('CN', 'JC', 'AA'))


fig1.savefig('Node2Vec_Indices.jpg')

## Training Score

Trial 1:

The auc score for dot product is:  0.5026712408650089

The auc score for elementwise mul:  0.5065504

The auc score for JC index:  0.5927655799999999

The auc score for AA index:  0.5989542999999999

The auc score for CN index:  0.59798122


Trial 2:

The auc score for dot product is:  0.5047348876223744

The auc score for elementwise mul:  0.49794066000000003

The auc score for JC index:  0.59610572

The auc score for AA index:  0.6006679199999999

The auc score for CN index:  0.60034756


Trial 3: 5000 epochs for LR training

The auc score for dot product is:  0.4985369616675093

The auc score for elementwise mul:  0.50202444

The auc score for JC index:  0.59311582

The auc score for AA index:  0.59906734

The auc score for CN index:  0.5984018799999999


In [ ]:
#alpha * train_loss + (1-alpha)*loss

    
#for e in range(20):
#    train_loss = 0
#    for i in range(len(left_ids)):
#        u = left_ids[i]
#        v = right_ids[i]
#        em = u * v
#        y_pred = link_model(em)
#        y_pred = torch.squeeze(y_pred)
#        optimizer_link.zero_grad() 
#        loss = criterion(y_pred, y_label_model[i])
#        train_loss += loss.item()
#        loss.backward()
#        optimizer_link.step()
#    print(train_loss)